## More about collections

### Sequences

Sequences (`seq`) are another type of collection that F# offers, and are commonly used when one has a large amount of data. In effect, unlike lists, the elements of a sequence are computed as they are required. On the other hand, sequences are associated with the ` IEnumerable<T>` type of the .NET ecosystem, making them suitable for occasions when one has to interact with libraries or interfaces written in other languages, such as C# or VB.

To construct a sequence, we use (ahem...) a `for... in ...do` constructor, as seen in the following examples:

In [1]:
let l = seq { for x in 1..5 -> x }
l

[ 1, 2, 3, 4, 5 ]

In [2]:
let sq =  seq {for x in 1..5 -> x*x }
sq

[ 1, 4, 9, 16, 25 ]

The use of the `->` arrow is a syntactic sweetener used when the expression evaluating each element of the sequence is simple. Otherwise, the `do...yield` constructor is used

In [3]:
let even = seq {for x in 0..3..30 do
                if (x % 2 = 0) then 
                    yield x}
even                   

[ 0, 6, 12, 18, 24, 30 ]

An important difference from lists is that it is not possible to access the elements of a sequence using the index in the usual way, such as `even[3]`

In [4]:
even[3]

Error: input.fsx (1,1)-(1,8) typecheck error The type 'IEnumerable<_>' does not define the field, constructor or member 'Item'.

Instead, use the `Seq.item` method or its `Seq.nth` equivalent:

In [5]:
Seq.item 3 even

18

In [6]:
Seq.nth 0 even

0

In [7]:
Seq.nth 14 even

Error: System.ArgumentException: The input sequence has an insufficient number of elements.
seq was short by 9 elements (Parameter 'index')
   at Microsoft.FSharp.Collections.Internal.IEnumerator.nth[T](Int32 index, IEnumerator`1 e) in D:\a\_work\1\s\src\FSharp.Core\seq.fs:line 43
   at Microsoft.FSharp.Collections.SeqModule.Item[T](Int32 index, IEnumerable`1 source) in D:\a\_work\1\s\src\FSharp.Core\seq.fs:line 613
   at <StartupCode$FSI_0012>.$FSI_0012.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

Complex sequences can be constructed with any of the F# data types:

In [8]:
type Person = { Name: string; Age: int }

In [49]:
let people = seq {
    { Name = "Alice"; Age = 25 }; { Name = "Bob"; Age = 30 }; { Name = "Charlie"; Age = 35 }}
let names = seq {
                    for person in people -> person.Name
                    }
printfn "%A" names 

seq ["Alice"; "Bob"; "Charlie"]


And use the `for` constructor in an imperative style to, for example, filter a sequence:

In [50]:
seq {for person in people do 
        if (person.Age < 33) then yield person.Name}

[ Alice, Bob ]

The `yield` constructor adds one element to the sequence, but when we want to add multiple elements, we use `yield!`. In the following example, we concatenate two sequences into a third

In [51]:
let joined = 
    seq {
        yield! l
        yield! sq
    }

printfn $"l: %A{l}"
printfn $"sq: %A{sq}"
printfn $"joined: %A{joined}"

l: seq [1; 2; 3; 4; ...]
sq: seq [1; 4; 9; 16; ...]
joined: seq [1; 2; 3; 4; ...]


> Note the use of the %A format descriptor in the interpolated string.

In [52]:
let anotherJoined = 
    seq {
        yield! l
        for x in 6..10 do x 
    }

anotherJoined

[ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 ]

#### A bit of `string` handling

Let's start using the full power of the .NET libraries:

- `System.IO.Path` which handles everything to do with _paths_, ie full filenames.
- `System.String` which contains methods for working with `strings`

In [53]:
let filepaths = seq { "file1.txt"; "file2.exe"; "file3.txt"; "file4.doc"; "file5.pdf" }

let extensions =
    seq {
        for filepath in filepaths do
            let extension = Path.GetExtension(filepath)
            if not (String.IsNullOrEmpty(extension)) then
                yield extension.Substring(1) // Grabs substring beginning at char index 1 
    }
    |> Seq.distinct

printfn "%A" extensions 

seq ["txt"; "exe"; "doc"; "pdf"]


The `.Substring(n)` method of an object of type `string` allows obtaining the string of characters included in said object that begins at the nth character

In [54]:
let titulo = "La bella y graciosa moza marchóse a lavar la ropa"

printfn $"{titulo}"
printfn $"{titulo.Substring(3)}"

La bella y graciosa moza marchóse a lavar la ropa
bella y graciosa moza marchóse a lavar la ropa


#### Lazy evaluation

As mentioned above, sequence elements are built as they are needed, in a process characteristic of many functional languages ​​called _lazy evaluation_:

In [12]:
let intSeq =
    seq { for n in 1 .. 10 do
            printfn "intSeq: %i" n
            yield n }
            
Seq.nth 3 intSeq
Seq.nth 5 intSeq
            

intSeq: 1
intSeq: 2
intSeq: 3
intSeq: 4
intSeq: 1
intSeq: 2
intSeq: 3
intSeq: 4
intSeq: 5
intSeq: 6


6

In [10]:
Seq.nth 5 intSeq


intSeq: 1
intSeq: 2
intSeq: 3
intSeq: 4
intSeq: 5
intSeq: 6


6

This feature allows methods like the following to exist, which "evaluates" an infinite sequence:

In [14]:
let allNumbers = Seq.initInfinite (fun i -> i * 2)

Over which I can take a certain number of elements with the `Seq.take` method

In [15]:
printfn "%A" (allNumbers |> Seq.take 20)
printfn "%A" (allNumbers |> Seq.take 20 |> Seq.length)
printfn "%A" (allNumbers |> Seq.take 30)
printfn "%A" (allNumbers |> Seq.take 30 |> Seq.length)

seq [0; 2; 4; 6; ...]
20
seq [0; 2; 4; 6; ...]
30


But you have to be careful when doing this kind of thing:

In [16]:
printfn "%A" (allNumbers |> Seq.length) // Ojo!!!!

Error: System.InvalidOperationException: Enumeration based on System.Int32 exceeded System.Int32.MaxValue.
   at Microsoft.FSharp.Collections.Internal.IEnumerator.upto@323.System.Collections.IEnumerator.MoveNext() in D:\a\_work\1\s\src\FSharp.Core\seq.fs:line 336
   at Microsoft.FSharp.Collections.SeqModule.Length[T](IEnumerable`1 source) in D:\a\_work\1\s\src\FSharp.Core\seq.fs:line 870
   at <StartupCode$FSI_0024>.$FSI_0024.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

The `seq` type is compatible with the `IEnumerable<'T>` type of the .NET ecosystem. Therefore, any use of a .NET library that returns an `IEnumerable<'T>`, is used directly as a sequence:

In [19]:

let cwd = System.IO.Directory.GetCurrentDirectory()
printfn "%s" cwd

let files =
        System.IO.Directory.EnumerateFiles(cwd)
        |> Seq.map(fun f -> (System.IO.Path.GetFileName(f),File.GetCreationTime(f)))
        

files 
|> Seq.iter(fun (name,date) -> printfn $"{name} created at {date}") 

files
|> Seq.toList        

/Users/flavioc/Library/Mobile Documents/com~apple~CloudDocs/Documents/Blog/es
IntroToTypes_es.ipynb created at 4/13/2023 10:15:44 AM
DiscriminatedUnions_es.ipynb created at 4/17/2023 9:39:43 AM
Fundamentals_es.ipynb created at 4/13/2023 1:51:14 PM
Results.ipynb created at 4/27/2023 2:27:32 PM
Exceptions.ipynb created at 4/25/2023 9:39:00 AM
YetAnotherTakeOnCollections.ipynb created at 5/1/2023 5:02:39 PM
IO.ipynb created at 5/2/2023 10:41:05 AM
OnCollections.ipynb created at 4/25/2023 9:39:00 AM
DUs.ipynb created at 4/17/2023 9:39:43 AM
ControlFlow.ipynb created at 4/13/2023 1:51:14 PM
Tuples.ipynb created at 4/19/2023 9:06:42 AM
MoreOnCollections.ipynb created at 4/25/2023 9:39:00 AM
Introduccion.md created at 11/3/2022 4:54:54 PM
Maps.ipynb created at 4/25/2023 9:39:00 AM
Records_es.ipynb created at 4/13/2023 10:16:13 AM
TypeExercises.ipynb created at 4/13/2023 11:11:06 AM
MasSobreFunciones.ipynb created at 4/13/2023 1:51:14 PM
Functions_es.ipynb created at 4/13/2023 1:51:14 PM
Units

index value 0 (IntroToTypes_es.ipynb, 4/13/2023 10:15:44 AM) Item1 IntroToTypes_es.ipynb Item2 2023-04-13 10:15:44Z 1 (DiscriminatedUnions_es.ipynb, 4/17/2023 9:39:43 AM) Item1 DiscriminatedUnions_es.ipynb Item2 2023-04-17 09:39:43Z 2 (Fundamentals_es.ipynb, 4/13/2023 1:51:14 PM) Item1 Fundamentals_es.ipynb Item2 2023-04-13 13:51:14Z 3 (Results.ipynb, 4/27/2023 2:27:32 PM) Item1 Results.ipynb Item2 2023-04-27 14:27:32Z 4 (Exceptions.ipynb, 4/25/2023 9:39:00 AM) Item1 Exceptions.ipynb Item2 2023-04-25 09:39:00Z 5 (YetAnotherTakeOnCollections.ipynb, 5/1/2023 5:02:39 PM) Item1 YetAnotherTakeOnCollections.ipynb Item2 2023-05-01 17:02:39Z 6 (IO.ipynb, 5/2/2023 10:41:05 AM) Item1 IO.ipynb Item2 2023-05-02 10:41:05Z 7 (OnCollections.ipynb, 4/25/2023 9:39:00 AM) Item1 OnCollections.ipynb Item2 2023-04-25 09:39:00Z 8 (DUs.ipynb, 4/17/2023 9:39:43 AM) Item1 DUs.ipynb Item2 2023-04-17 09:39:43Z 9 (ControlFlow.ipynb, 4/13/2023 1:51:14 PM) Item1 ControlFlow.ipynb Item2 2023-04-13 13:51:14Z 10 (Tuples.ipynb, 4/19/2023 9:06:42 AM) Item1 Tuples.ipynb Item2 2023-04-19 09:06:42Z 11 (MoreOnCollections.ipynb, 4/25/2023 9:39:00 AM) Item1 MoreOnCollections.ipynb Item2 2023-04-25 09:39:00Z 12 (Introduccion.md, 11/3/2022 4:54:54 PM) Item1 Introduccion.md Item2 2022-11-03 16:54:54Z 13 (Maps.ipynb, 4/25/2023 9:39:00 AM) Item1 Maps.ipynb Item2 2023-04-25 09:39:00Z 14 (Records_es.ipynb, 4/13/2023 10:16:13 AM) Item1 Records_es.ipynb Item2 2023-04-13 10:16:13Z 15 (TypeExercises.ipynb, 4/13/2023 11:11:06 AM) Item1 TypeExercises.ipynb Item2 2023-04-13 11:11:06Z 16 (MasSobreFunciones.ipynb, 4/13/2023 1:51:14 PM) Item1 MasSobreFunciones.ipynb Item2 2023-04-13 13:51:14Z 17 (Functions_es.ipynb, 4/13/2023 1:51:14 PM) Item1 Functions_es.ipynb Item2 2023-04-13 13:51:14Z 18 (Units.ipynb, 4/27/2023 2:27:32 PM) Item1 Units.ipynb Item2 2023-04-27 14:27:32Z

Here we use .NET modules

- `System.IO.Directory` which handles everything that has to do with directories, ie full filenames.
- `System.IO.File` which contains methods for working with files.

and the `Seq.map` and `Seq.iter` methods similar to those used with lists.

### Lists comprehension

Similar to the preceding paragraphs, where we used the `[for x in collection do ... yield expr]` to recreate sequences; and just like other languages ​​like [Python](https://www.w3schools.com/python/python_lists_comprehension.asp) or [Haskell](https://wiki.haskell.org/List_comprehension) construct, it can be used in F# to perform _list compressions_, that is, an imperative way of constructing a list.

In [60]:
let l = [for x in 1..5 -> x ]
l.Head

1

In [61]:
let sq =  [for x in 1..5 -> x*x ]
sq

[ 1, 4, 9, 16, 25 ]

In [62]:
let even = [for x in 0..3..30 do
                if (x % 2 = 0) then 
                    yield x]
even                     

[ 0, 6, 12, 18, 24, 30 ]

In [63]:
type Person = { Name: string; Age: int }

In [64]:
let people = [{ Name = "Alice"; Age = 25 }; { Name = "Bob"; Age = 30 }; { Name = "Charlie"; Age = 35 }]
let names = [for person in people -> person.Name]
printfn "%A" names 

["Alice"; "Bob"; "Charlie"]


In [65]:
[for person in people do 
    if (person.Age < 33) then yield person.Name]

[ Alice, Bob ]

Note that lists are not lazily evaluated:

In [66]:
let intLst =
    [ for a in 1 .. 10 do
        printfn "intLst: %i" a
        yield a ]

intLst: 1
intLst: 2
intLst: 3
intLst: 4
intLst: 5
intLst: 6
intLst: 7
intLst: 8
intLst: 9
intLst: 10
